In [104]:
import numpy as np
import pandas as pd
import string , spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [105]:
#load data
df = pd.read_csv('ar_reviews_100k.tsv', sep=('\t'))
df.head()


,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [106]:
print("shape before any preprocessing " , df.shape)

shape before any preprocessing  (99999, 2)


In [107]:
#Drop missing rows
df = df.dropna()

In [108]:
df.isnull()   #check missing values in each row

,label,text
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
99994,False,False
99995,False,False
99996,False,False
99997,False,False


In [109]:
df.isna().sum()    #checksum of missing values in my file

label    0
text     0
dtype: int64

In [110]:
df['label'].value_counts()

Positive    33333
Mixed       33333
Negative    33333
Name: label, dtype: int64

In [111]:
### convert positive 1 and negative 0
df['label']=df['label'].apply(lambda x: 1 if x=='Positive' else 0 )
df.head()

,label,text
0,1,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,1,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,1,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,1,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [112]:
df['label'].value_counts()

0    66666
1    33333
Name: label, dtype: int64

In [113]:
print('total shape after removing mixed label from data ')
df.shape

total shape after removing mixed label from data 


(99999, 2)

In [114]:
#check on duplicated text
m=df.duplicated().sum()
m
if m==0 :
   print("all values are unique")

all values are unique


In [115]:
import nltk


words1=['أ','د','ج','ح','خ','ه','ع','غ','ف','ق','ث','ص','ض','ذ','ش','س','ي','ب'
,'ل','ا','ت','ن','م','ك','ط','ظ','ز','و','ة','ى','ر','ؤ','ء','ئ','إنما', 'هؤلاء', 'أربعمئة', 'ى', 'هيّا', 'نحو'
, 'كيف', 'جويلية', 'أنا', 'كليهما', 'قام', 'ث' , 'جيم', 'أولئك', 'والذي', 'ولكن', 'عشر', 'ف', 'مايو', 'ع',  'يفعلان'
, 'أغسطس', 'عن', 'مما', 'نفس', 'أنتِ', 'بماذا', 'تموز', 'إياكما', 'قطّ', 'حيَّ', 'بات', 'مكانكم', 'ثمانية', 'مع', 'ثمّ', 'هو'
, 'درى', 'كاد', 'وهو', 'إيه', 'عدَّ', 'مه', 'إذ', 'ليسوا', 'واحد',  'أين', 'لستما', 'رويدك', 'لها', 'إمّا', 'آب', 'ترك', 'حار'
, 'فبراير', 'خمسمائة', 'تسع', 'ظنَّ', 'إليكنّ', 'تينك', 'نون', 'أول', 'على', 'وا', 'كذا', 'انبرى', 'دون', 'لك', 'ذيت', 'ثلاثين'
, 'غادر', 'ذِه', 'أعطى', 'لات', 'عليك', 'ثمان', 'ذه', 'وما', 'ذا', 'لولا', 'ثلاث', 'ذ', 'اثنين', 'ثلاثمائة', 'هيت', 'ذانِ'
, 'أيا', 'جير', 'عاشر', 'إياك', 'سقى', 'أنت', 'كأنّ', 'كرب', 'سبعمئة', 'وجد', 'فضلا', 'ذِي', 'خمسة', 'أبريل', 'اللذان'
, 'تلقاء', 'كأنما', 'كِخ', 'إما', 'ظاء', 'بمن', 'كأيّ', 'سبع', 'أُفٍّ', 'أمسى', 'لا سيما', 'آهٍ', 'بكن', 'نحن', 'هاته', 'ذوا'
, 'خاصة', 'ذينك', 'كلَّا', 'بئس', 'إياي', 'هكذا', 'ت', 'خ', 'ثمانمئة', 'هاكَ', 'أخٌ', 'فإن', 'ته', 'سوف', 'إي', 'أربع'
, 'قبل', 'هَيْهات', 'خبَّر', 'أولاء', 'حيث', 'ذواتي', 'يمين', 'كما', 'ها', 'ثمنمئة', 'كاف', 'سنتيم', 'أل', 'ستمائة', 'ا', 'أ'
, 'مكانَك', 'لكي', 'تلكما', 'عاد', 'الآن', 'ذلكم', 'فيما', 'إليكن', 'ص', 'سبت', 'صاد', 'إياه', 'لهم', 'أعلم', 'تجاه', 'ستين'
, 'حاء', 'لكيلا', 'إلى', 'أمامك', 'عشرين', 'ألفى', 'ثلاثة', 'هَذانِ', 'ين', 'بَلْهَ', 'شبه', 'باء', 'هَؤلاء', 'بهما', 'عل', 'فو'
, 'أخبر', 'يا', 'صبر', 'تلكم', 'متى', 'أقل', 'بعدا', 'ابتدأ', 'هن', 'أنتن', 'ئ', 'عشرة', 'ماي', 'استحال', 'ذان', 'أيار'
, 'مكانكما', 'دواليك', 'عَدَسْ', 'ليست', 'ممن', 'ضاد', 'آ', 'عين', 'هذي', 'ميم', 'صار', 'ما انفك', 'ن', 'اخلولق', 'ألف'
, 'رأى', 'أفٍّ', 'تسعمائة', 'لام', 'أخو', 'أخذ', 'ريث', 'حمدا', 'كيت', 'ءَ', 'أبو', 'حَذارِ', 'كيفما', 'كانون', 'ما برح'
, 'مارس', 'يوان', 'تشرين', 'غين', 'فاء', 'ما أفعله', 'لستم', 'ياء', 'طالما', 'كان', 'هذان', 'خال', 'هي', 'خاء', 'ذات'
, 'ثان', 'هَاتِي', 'لعلَّ', 'و', 'فإذا', 'إليكما', 'هذا', 'واو', 'ج', 'بين', 'مافتئ', 'شيكل', 'عوض', 'إلَيْكَ', 'معاذ', 'لوما'
, 'تسعين', 'ذاك', 'بكم', 'ثاء', 'إذن', 'بَسْ', 'شرع', 'سبعة', 'جنيه', 'كأيّن', 'فلان', 'مازال', 'نَّ', 'بكما', 'له', 'إى'
, 'عيانا', 'بخٍ', 'نوفمبر', 'ستون', 'لن', 'طاق', 'درهم', 'ثمّة', 'كلتا', 'إياكن', 'ه', 'لهن', 'اللتين', 'التي', 'دولار', 'لنا'
, 'ط', 'إيهٍ', 'خمسين', 'ليس', 'تبدّل', 'ذي', 'هاهنا', 'أكتوبر', 'اللائي', 'سبتمبر', 'لدى', 'إياها', 'تاء', 'صبرا', 'راح'
, 'مائة', 'آناء', 'تِي', 'في', 'أقبل', 'جعل', 'تعلَّم',  'أرى', 'اللواتي', 'ق', 'واهاً', 'بلى', 'فيها', 'أمّا', 'ثمانون', 'نبَّا'
, 'هبّ',  'حبذا', 'أو', 'هم', 'هَذَيْنِ', 'هل', 'آذار', 'منها', 'خميس', 'أهلا', 'نا', 'كذلك', 'لكم', 'خلافا', 'أنشأ', 'ديسمبر'
, 'سبعين', 'شتان', 'ذلكما', 'إليكم', 'لو', 'من', 'تِه', 'صهٍ', 'عسى', 'هلا', 'تسعون', 'هَاتِه', 'فيه', 'أمد', 'هَاتانِ', 'إن'
, 'ذهب', 'ليت', 'شين', 'ل', 'هاك', 'جميع', 'هَجْ', 'أمام', 'غ', 'ذانك', 'عشرون', 'كلاهما', 'سرا', 'ح', 'سرعان'
, 'رزق', 'علم', 'ساء', 'خمس', 'إليك', 'يونيو', 'حسب', 'منذ', 'أيلول', 'لاسيما', 'ء', 'سابع', 'أربعاء', 'فلس', 'زاي'
, 'حرى', 'بي', 'قرش', 'زود', 'نعم', 'يورو', 'اللاتي', 'لستن', 'منه', 'حدَث', 'صراحة', 'رابع', 'ظ', 'أجل', 'أى'
, 'هلّا', 'كأي', 'حجا', 'لسن', 'هَذِي', 'سوى', 'أنتما', 'دونك', 'طرا', 'به', 'أيها', 'سادس', 'مثل', 'وإن', 'أنتم'
, 'أما', 'بك', 'إنَّ', 'زعم', 'لكن', 'تفعلين', 'ما', 'لما', 'آمينَ', 'يوليو', 'جوان', 'أنى', 'تانِ', 'س', 'كى', 'إذا'
, 'ر', 'تخذ', 'ثاني', 'حادي', 'قد', 'بس', 'غالبا', 'كلما', 'بهم', 'حتى', 'أم', 'أربعة', 'تلك', 'اثني', 'أف', 'تي'
, 'قاطبة', 'هذين', 'كأين', 'هنا', 'همزة', 'خلا', 'ست', 'أيّان', 'شمال', 'إذما', 'ثمانين', 'تحت', 'هذه', 'نَخْ', 'بعض'
, 'ظلّ', 'خامس', 'فمن', 'أبٌ', 'وإذ', 'شتانَ', 'غير', 'لئن', 'أمس', 'إليكَ', 'كل', 'اللتيا', 'مهما', 'خمسون', 'أيّ', 'ذو'
, 'اربعين', 'بما', 'قاف', 'ز', 'ؤ', 'ضحوة', 'غداة', 'وراءَك', 'رجع', 'راء', 'بضع', 'أوه', 'بل', 'بسّ', 'تين', 'ثامن'
, 'أمامكَ', 'إنا', 'سبحان', 'علق', 'تفعلان', 'مساء', 'بهن', 'حاي', 'جلل', 'وإذا', 'أجمع', 'ذلك', 'عجبا', 'ب', 'بيد'
, 'ليستا', 'بعد', 'أضحى', 'مليم', 'الذين', 'اثنا', 'أصلا', 'ومن', 'ذَيْنِ', 'تعسا', 'أولالك', 'سين', 'كلا', 'لست'
, 'عامة', 'كليكما', 'هَذِه', 'آهاً', 'ثمة', 'حاشا', 'أنبأ', 'اثنان', 'دينار', 'علًّ', 'ولو', 'إذاً', 'لعمر', 'تحوّل'
, 'لكنما', 'فيم', 'حمٌ', 'ّأيّان', 'حيثما', 'ليرة', 'آض', 'هاء', 'كم', 'اتخذ', 'الذي', 'انقلب', 'يفعلون',  'ثلاثاء'
, 'خمسمئة', 'عدا', 'هناك', 'اربعون', 'أنّى', 'ماذا', 'طاء',  'مرّة'
, 'هَذا', 'ثالث', 'إحدى', 'آهِ', 'وَيْ', 'مئتان', 'كثيرا', 'آي'
, 'مذ', 'نيف', 'شَتَّانَ', 'قلما', 'أصبح', 'أسكن', 'الألى', 'يناير'
, 'صباح', 'ذواتا', 'ارتدّ', 'سبعون', 'غدا', 'إياهم', 'دال', 'فيفري'
, 'مادام', 'لمّا', 'رُبَّ', 'ريال', 'لي', 'شباط', 'ورد', 'نيسان'
, 'سبعمائة', 'ذال', 'خلف', 'هما', 'ستمئة', 'أيضا', 'فرادى', 
'تارة', 'والذين', 'آه', 'مكانكنّ', 'طَق', 'أفعل به', 'بنا', 'إيانا'
, 'ة', 'لعل', 'لكنَّ', 'اللذين', 'هللة', 'أوت', 'جمعة', 'إياكم', 'أوّهْ'
, 'كلّما', 'د', 'فوق', 'أي', 'لكما', 'ش', 'إنه', 'حزيران', 'الألاء'
, 'جانفي', 'بها', 'هاتي', 'تَيْنِ', 'عند', 'لهما', 'بغتة', 'عليه', 'ذلكن', 'ثمَّ'
, 'أينما', 'اللتان', 'كأن', 'بخ', 'هلم', 'كسا', 'تسعمئة', 'هيا', 'هيهات'
, 'م', 'هنالك', 'بطآن', 'طفق', 'حقا', 'حين', 'أفريل', 'أربعمائة', 'حمو'
, 'مئة', 'كي', 'لدن', 'إزاء', 'عما', 'تانِك', 'آنفا', 'ثلاثمئة', 'أكثر'
, 'أوشك', 'إياهما', 'وهب', 'آها', 'لبيك', 'هاتين', 'تسعة', 'هَاتَيْنِ'
, 'أنًّ', 'وُشْكَانَ', 'سحقا', 'ك', 'صهْ', 'ثم', 'سمعا', 'ض'
, 'ي', 'صدقا', 'ثماني', 'ثلاثون', 'تفعلون', 'بؤسا', 'هاتان'
, 'إياهن', 'ذين', 'ستة', 'أن', 'لسنا', 'تاسع', 'كن', 'أحد']

words2= ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم'
, 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل'
, 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد'
, 'قط', 'لا', 'لم', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها'
, 'وراء' , 'أيان','يان', 'بله', 'تانك', 'تبدل', 'تحول', 'تعلم', 'حدث', 'حذار', 'خبر', 'رب', 'طق', 'عدس', 'كخ', 'مرة', 'مكانك', 'نبا', 'نخ', 'هج', 'وراءك', 'وشكان', 'وي']

stopwords =words1 + words2

stop_word = list(set(stopwords))        #make stop unique




print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print('stop_words shape is ',len(stopwords))
print('stop_words shape without duplicated words ',len(stop_word))

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
stop_words shape is  785
stop_words shape without duplicated words  751


In [116]:
import re
import string
import sys
import argparse
from nltk.tokenize import word_tokenize

arabic_punctuations = '''`÷×~؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations = arabic_punctuations + english_punctuations
punctuations

'`÷×~؛<>_()*&^%][ـ،/:"؟.,\'{}~¦+|!”…“–ـ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [117]:
def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', str(text))
    return text

In [118]:
from nltk.stem import WordNetLemmatizer

def process_text(text):

    stemmer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    #stemming
    word_list = [stemmer.lemmatize(word) for word  in  word_list]
    return ' '.join(word_list) 

In [119]:
def preprocessing(text):
  
    text = "".join([word for word in text if word not in string.punctuation])
    text = process_text(text)
    text = remove_diacritics(text)
    tokens = word_tokenize(text)
    text = ' '.join([word for word in tokens if word not in stop_word])
    return text

In [120]:
for i in range(0,5):
  m= preprocessing(df['text'][i])
  print(i, ")", m, end='\n')

0 ) ممتاز نوعا النظافة والموقع والتجهيز والشاطيء المطعم
1 ) أسباب نجاح الإمارات شخص الدولة يعشق ترابها نحب الإمارات ومضات فكر نصائح لدولة تطمح بالصفوف الأولى قائد يقبل إلا براحة شعبه وتوفر سب العيش الكريم حكم مواقف ونصائح لكل فرد فينا بمجرد كتاب سياسي كنت اعتقد يستحق القراءة مرات كثيرة
2 ) هادفة وقوية تنقلك صخب شوارع القاهرة الى هدوء جبال الشيشان للتعرف حقيقة يجرى فى البلاد حروب ضاربة بحق المسلمين جزء كبير تاريخ المنطقة التضحية الرجولة الوفاء والكثير القيم الأخرى اثبتت وجودها فى الرواية البسيطة
3 ) خلصنا مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن احمد مراد تخطى مرحلة القارئ يخلص الرواية فاتح بؤه لمرحلة القارئ يخلص الرواية محترم الكاتب اتقان مخيف بصرف النظر اخطاء تذكر الحوار انما احمد مراد سافر عاش حبة اوائل القرن العشرين وجه ياخدنا لهناك خلطة مشاعر انسانية حقيقية لدرجة غريبة دراما نقلته كاتب شاب بيستعرض لصنايعي حقيقي تثق سيكتب كده ربط مصر ومصر سنة هيطمنك عجينة الناس دي متغيرتش زي عدت كده هتعدي اي ازمة دلوقتي احمد مراد كاتب محترف شغال بضميييررر بكل تحمل الكلمة معنى
4 ) ياسات جل

In [121]:
nltk.download('omw-1.4')
X =df['text']	
Y =df['label']	

#delete stopwords and most frequent words and less used words
tfidfconverter= TfidfVectorizer( min_df=4, max_df=0.9, stop_words=stop_word, preprocessor=preprocessing)
X_Vec = tfidfconverter.fit_transform(X)
X_Vec = pd.DataFrame.sparse.from_spmatrix(X_Vec)

print(f'The old shape for X is {X.shape}')
print(f'The new shape for X is {X_Vec.shape}')
X_Vec.head()


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


The old shape for X is (99999,)
The new shape for X is (99999, 78867)


,0,1,2,3,4,5,6,7,8,9,...,78857,78858,78859,78860,78861,78862,78863,78864,78865,78866
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [123]:
X_train_tfidf = tfidfconverter.fit_transform(X_train) 
X_test_tfidf = tfidfconverter.fit_transform(X_test) 
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(66999, 60723)
(33000, 37048)


In [124]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC()

In [125]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [126]:


from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))



[[19382  2685]
 [ 3902  7031]]


In [127]:
print(metrics.classification_report(y_test,predictions)) 

              precision    recall  f1-score   support

           0       0.83      0.88      0.85     22067
           1       0.72      0.64      0.68     10933

    accuracy                           0.80     33000
   macro avg       0.78      0.76      0.77     33000
weighted avg       0.80      0.80      0.80     33000



In [128]:
print(metrics.accuracy_score(y_test,predictions))

0.8003939393939394
